In [8]:
import pandas as pd
import missingno

In [9]:
df1 = pd.read_csv('log_film_0601_0605.csv')
df2 = pd.read_csv('log_film_0605_0610.csv')
df3 = pd.read_csv('log_film_0610_0615.csv')
df4 = pd.read_csv('log_film_0615_0620.csv')
df5 = pd.read_csv('log_film_0620_0625.csv')

train_df = pd.concat([df1,df2,df3,df4,df5])
train_df.dropna(inplace=True)

/home/admin1/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3221: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/admin1/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3221: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
film_espisode = pd.read_csv('tv360_film_espisode.csv')
film_series = pd.read_csv('tv360_film_series.csv')
episode2series=dict(zip(film_espisode.episode_id,film_espisode.series_id))
series_id_ls = film_series.series_id.unique()
episode_id_ls = film_espisode.episode_id.unique()

In [46]:
train_df.dtypes

user_id                 object
profile_id             float64
content_id              object
watch_duration         float64
lognum                   int64
pause_count              int64
seek_count               int64
change_volumn_count      int64
partition                int64
series_id              float64
dtype: object

In [13]:
def convert_series_id(x,series_id_ls,episode_id_ls,episode2series):
    if x in series_id_ls:
        return x
    elif x in episode_id_ls:
        return episode2series[x]

In [19]:
train_df['series_id'] = train_df.content_id.apply(lambda x: convert_series_id(x))

/tmp/ipykernel_3241776/860769408.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if x in series_id_ls:
/tmp/ipykernel_3241776/860769408.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif x in episode_id_ls:


In [23]:
train = train_df[['user_id','series_id','watch_duration']]

In [24]:
x = train.groupby(['user_id','series_id'])['watch_duration'].sum()

x

user_id   series_id
0.0       851.0         30.0
          1022.0        30.0
          1289.0        30.0
          1349.0       240.0
          1406.0       660.0
                       ...  
99999910  9361.0        77.0
99999937  15253.0        0.0
          15310.0        0.0
99999940  6546.0         0.0
          6787.0         6.0
Name: watch_duration, Length: 2385071, dtype: float64

In [29]:
train = pd.DataFrame(data =list(x.index.values), columns=['user_id','series_id'] )
train['watch_duration']= list(x.values)

In [31]:
def rating(x):
    if x >300:
        return 5
    else :
        return x//60

In [32]:
train['rate'] = train.watch_duration.apply(lambda x: rating(x))

In [36]:
out = train[['user_id','series_id','rate','watch_duration']]

In [57]:
out = out[out['user_id']!='""']

In [63]:
out =out.astype('int')

In [60]:
out['user_id'] = out['user_id'].apply(lambda x: int(float(x)))

/tmp/ipykernel_3241776/1357139521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out['user_id'] = out['user_id'].apply(lambda x: int(float(x)))


In [64]:
out

,user_id,series_id,rate,watch_duration
0,0,851,0,30
1,0,1022,0,30
2,0,1289,0,30
3,0,1349,4,240
4,0,1406,5,660
...,...,...,...,...
2385066,99999910,9361,1,77
2385067,99999937,15253,0,0
2385068,99999937,15310,0,0
2385069,99999940,6546,0,0


In [65]:
out.to_csv('train.rating',index=False,header=False)